```python
# Copyright 2022 Bloomberg Finance L.P.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

# Market Value Item

### What is market value?

All of the previously discussed items have focused on different collections of securities (positions, accounts, and portfolios). Now we're going to take another step forward and discuss market value. Market value is generally the price that an asset of any type (e.g., equity, fixed income, derivatives, fx) would fetch in the marketplace. It is also the total value of the securities in which you hold a position. Depending on the security type, market value can be simple (i.e., for equities, the value of a unit is well-known) or more complex with various options (i.e., for derivatives, there are a few ways to determine and/or state what the market value of your position is).

For our purpose we'll stick with simple market value calculations where market value is equal to some unit multiplier times your positions

For example, you could have a position and current share price/unit market value like below:

My position is ***2000*** shares of ***MSFT USD***. The current price of ***MSFT USD*** is ***19.00***. This gives my position a 
market value of **38,000 USD**!

### Problem Definition

We want to extend our classes to allow for querying of the market values for various securities at different levels of aggregation. We want this classes to have to the ability below functionality:

***Security***
- Query the current market value of the security

***Positions*** 
- Query the current market value for the current position

***Account***
- Query the market value for a given set of security names or security objects. These objects should act as filters to what should be included in calculating the accounts net market value. Any securities not present in the account but the filter can be ignored
- Query the market value for the entire account. This should return a net market value of all positions in the account.

***Portfolio***
- Query the market value for the entire portfolio. This requires that security market values be atomic in your calculation method per query.
- Query the market value for a given set of securities/account combinations. If securities are present they should act as a filter on the accounts to be queried. If accounts are present they should act as a filter on the accounts under query. Handle scenarios where securities or accounts given aren't present without causing exceptions.
- **Create tests for the above portfolio features**

### Provided Tools

#### *Data Source*

To simulate prices, you can utilize the given class priceData which can be imported from generators.priceDataGenerator. This class is a singleton and will generate prices each time a query is given. These prices are stored internally in the class for testing in a replay mode. For a given security if the security name contains "Equity" or "eqty" (case insensitive) the market value will always be positive. To query the price of a security you can use the object's getCurrentPrice method. Below is an example of this

```python
#Data Souce
from generators.priceDataGenerator import priceData

pD = priceData()

curPrice = pD.getCurrentPrice("IBM US Equity")
```

#### *Solution Interface*

The below interface show the new functionality you should add to your existing classes.

```python
#filename securityInterface.py
#Security Class Interface
class securityInterface():
    #...
    #Previous implementation above
    def getCurrentMarketValue(self) -> float:
```

```python
#filename positionInterface.py
#Position Class Interface
class positionInterface():
    #...
    #Previous implementation above
    def getCurrentMarketValue(self) -> float:
        pass
```

```python
#filename accountInterface.py
#Account Class Interface
class accountInterface():
    #...
    #Previous implementation above
    
    def getCurrentMarketValue(self) -> float:
        pass

    def getCurrentFilteredMarketValue(self, securities: Set) -> float:
        pass
```

```python
#filename portfolioInterface.py
#Portfolio Class Interface
class portfolioInterface():
    #...
    #Previous implementation above
    
    def getCurrentMarketValue(self) -> dict:
        pass

    def getCurrentFilteredMarketValue(self, securities: Set, accountNames: Set[str]) -> float:
        pass
```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to files to their respective .py's in the implementation folders. **Remember to copy the current implementation as saving will overwrite your previous implementation!** 

In [30]:
%%writefile ../implementations/securitySolution.py
#Uncomment line above & run cell to save solution
#TODO Define class that implements securityInterface & allows for the management of a security

from PortfolioManager.interfaces.securityInterface import securityInterface
from PortfolioManager.generators.priceDataGenerator import priceData

pD = priceData()

class security(securityInterface):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self.name = name

    #Return the security's name
    def getName(self) -> str:
        return self.name

    #Return the current security's market value
    def getCurrentMarketValue(self) -> float:
        return pD.getCurrentPrice(self.name)

    def __str__(self):
        return f'(name={self.name}, value=None)'


Overwriting ../implementations/securitySolution.py


In [31]:
%%writefile ../implementations/positionSolution.py
#Uncomment line above & run cell to save solution
#TODO Define class that implements positionInterface & allows for the management of a position
from PortfolioManager.implementations.securitySolution import security
from PortfolioManager.interfaces.positionInterface import positionInterface
from PortfolioManager.interfaces.securityInterface import securityInterface

class position(positionInterface):
    def __init__(self, _security, initialPosition: int) -> None:
        if isinstance(_security, str):
            self.security = security(_security)
        else:
            self.security = _security
        self.setPosition(initialPosition)

    def getSecurity(self) -> securityInterface:
        return self.security

    def getPosition(self) -> int:
        return self.position

    def setPosition(self, inputValue: int) -> None:
        if inputValue < 0:
            raise Exception('Input value cannot be negative.')
        self.position = inputValue

    def addPosition(self, inputValue: int) -> None:
        if self.position + inputValue < 0:
            raise Exception('Input value cannot be negative.')
        self.position += inputValue

    def getCurrentMarketValue(self):
        return self.security.getCurrentMarketValue() * self.position


Overwriting ../implementations/positionSolution.py


In [36]:
%%writefile ../implementations/accountSolution.py
#Uncomment line above & run cell to save solution
#TODO Define class that implements accountInterface & allows for the management of an account

from PortfolioManager.interfaces.positionInterface import positionInterface
from typing import Set, Iterable, Dict, Any
from PortfolioManager.interfaces.accountInterface import accountInterface
from PortfolioManager.interfaces.securityInterface import securityInterface

class account(accountInterface):
    def __init__(self, positions: Set[positionInterface], accountName: str):
        super().__init__(positions, accountName)
        self.name = accountName
        self.positions = {pos.getSecurity().getName(): pos for pos in positions}

    def getName(self) -> str:
        return self.name

    def getAllPositions(self) -> Iterable[positionInterface]:
        return self.positions.values()

    def getPositions(self, securities: Set) -> Dict[Any, positionInterface]:
        return {pos.getSecurity().getName() : pos.getSecurity() for pos in self.positions
                if pos.getSecurity().getName() in [x.getName() for x in securities]}

    def addPositions(self, positions: Set[positionInterface]) -> None:
        for position in positions:
            self.positions[position.getSecurity().getName()] = position

    def removePositions(self, securities: Set) -> None:
        for security in securities:
            if security.getName() in self.positions:
                del self.positions[security.getName()]

    def getCurrentFilteredMarketValue(self, securitiesList):
        value = 0

        if securitiesList is not None and len(securitiesList) > 0:
            securitiesList = [sec.getName() if isinstance(sec, securityInterface) else sec for sec in securitiesList]
            for position in self.positions.values():
                if position.getSecurity().getName() not in securitiesList:
                    continue
                value += position.getCurrentMarketValue()
        else:
            for position in self.positions.values():
                value += position.getCurrentMarketValue()

        return value

    def getCurrentMarketValue(self):
        return self.getCurrentFilteredMarketValue([pos.getSecurity().getName() for pos in self.positions.values()])


Overwriting ../implementations/accountSolution.py


In [37]:
%%writefile ../implementations/portfolioSolution.py
#Uncomment line above & run cell to save solution
#TODO Define class that implements portFolioInterface & allows for the management of a portfolio

from typing import Set, Iterable
from PortfolioManager.interfaces.accountInterface import accountInterface
from PortfolioManager.interfaces.securityInterface import securityInterface
from PortfolioManager.interfaces.portfolioInterface import portfolioInterface

class portfolio(portfolioInterface):
    def __init__(self, portfolioName: str, accounts: Set[accountInterface]) -> None:
        self.name = portfolioName
        self.accounts = {account.getName() : account for account in accounts}

    def getName(self):
        return self.name

    def getAllAccounts(self) -> Iterable[accountInterface]:
        return self.accounts.values()

    def getAccounts(self, accountNamesFilter:Set[str], securitiesFilter:Set) -> Iterable[accountInterface]:
        ret = [account for account in self.accounts.values()]

        if accountNamesFilter is not None and len(accountNamesFilter) > 0:
            ret = [account for account in ret if account.getName() in accountNamesFilter]

        if securitiesFilter is not None and len(securitiesFilter) > 0:
            ret = [account for account in ret if self._allMatches(account, securitiesFilter) ]

        print(accountNamesFilter, securitiesFilter, [account.getName() for account in ret])
        return ret

    def _allMatches(self, account, securitiesFilter):
        securities = [position.getSecurity().getName() for position in account.getAllPositions()]
        for name in securitiesFilter:
            if name not in securities:
                return False
        return True

    def addAccounts(self, accounts: Set[accountInterface]) -> None:
        for account in accounts:
            self.accounts[account.getName()] = account

    def removeAccounts(self, accountNames: Set[str]) -> None:
        for account in accountNames:
            if account in self.accounts:
                del self.accounts[account]

    def getCurrentMarketValue(self):
        cache = {}

        value = 0
        for account in self.accounts.values():
             for position in account.getAllPositions().values():
                security = position.getSecurity()
                name = security.getName()
                if name not in cache:
                    cache[name] = security.getCurrentMarketValue()
                value += cache[name] * position.getPosition()
        return value

        pass

Overwriting ../implementations/portfolioSolution.py


In [38]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [39]:
%%ipytest -qq
import pytest
from PortfolioManager.implementations import securitySolution
from PortfolioManager.implementations import positionSolution
from PortfolioManager.implementations import accountSolution
from PortfolioManager.generators.priceDataGenerator import priceData
import importlib
importlib.reload(securitySolution)
importlib.reload(positionSolution)
importlib.reload(accountSolution)

def test_securityValueGather():
    #GIVEN
    SECURITY_NAME = "TSLA US Equity"
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()

    #WHEN
    testObj = securitySolution.security(SECURITY_NAME)
    currentPrice = testObj.getCurrentMarketValue()

    #EXPECT
    assert currentPrice == DATA_SOURCE.getSecurityPriceDataList(SECURITY_NAME)[-1]

def test_PositionMarketValue():
    #GIVEN
    EXPECTED_NAME = "IBM US Equity"
    EXPECTED_POSITION_AMOUNT = 1000
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()

    #WHEN
    testObj = positionSolution.position(EXPECTED_NAME, EXPECTED_POSITION_AMOUNT)
    MV = testObj.getCurrentMarketValue()
    LASTEST_EXPECTED_MV = EXPECTED_POSITION_AMOUNT * DATA_SOURCE.getSecurityPriceDataList(EXPECTED_NAME)[-1]

    #EXPECT
    assert (LASTEST_EXPECTED_MV == MV)

def test_SecuritySearchAccountMV():
    #GIVEN
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()
    EXPECTED_ACCOUNT_POSITIONS = [
        positionSolution.position("IBM US Equity", 530),
        positionSolution.position("TSLA US Equity", 1120),
        positionSolution.position("NVDA US Equity", 7421)
    ]
    SEARCH_SECURITIES_LIST =  ["IBM US Equity", securitySolution.security("NVDA US Equity"), "MSFT US Equity"]
    SEARCH_SECURITIES_TUPLE = [["IBM US Equity", 530], ["NVDA US Equity", 7421]]
    testObj = accountSolution.account(EXPECTED_ACCOUNT_POSITIONS, "Test Account")

    #WHEN
    MV = testObj.getCurrentFilteredMarketValue(SEARCH_SECURITIES_LIST)
    EXPECTED_MV = 0
    for secTuple in SEARCH_SECURITIES_TUPLE:
        if secTuple[1] != 0:
            EXPECTED_MV +=  secTuple[1] * DATA_SOURCE.getSecurityPriceDataList(secTuple[0])[-1]
    #EXPECT
    assert (EXPECTED_MV == MV)

def test_TotalAccountMV():
    #GIVEN
    EXPECTED_ACCOUNT_POSITIONS = [
        positionSolution.position("IBM US Equity", 530),
        positionSolution.position("TSLA US Equity", 1120),
        positionSolution.position("NVDA US Equity", 7421)
    ]
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()
    testObj = accountSolution.account(EXPECTED_ACCOUNT_POSITIONS, "Test Account")

    #WHEN
    MV = testObj.getCurrentMarketValue()
    EXPECTED_MV = 0
    for pos in EXPECTED_ACCOUNT_POSITIONS:
        EXPECTED_MV += pos.getPosition() * DATA_SOURCE.getSecurityPriceDataList(pos.getSecurity().getName())[-1]
    #EXPECT
    assert (EXPECTED_MV == MV)

#TODO Portfolio MV Tests

....                                                                                         [100%]
